In [ ]:
! pip install transformers datasets py7zr evaluate rouge_score

In [ ]:
from random import randrange

from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import pipeline
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Dataset

We are going to fine-tune the [Flan-T5](https://huggingface.co/google/flan-t5-small) model to summarize dialogue/ chat threads similar to what is done in the ChatGPT interface. We will use the [SAMSum](https://huggingface.co/datasets/samsum) dataset. The SAMSum dataset contains about 16k messenger-like conversations with summaries. Conversations were created and written down by linguists fluent in English.

(Thanks to [Philipp Schmid](https://www.philschmid.de/) for great examples of this task that were adapted for this notebook)

In [ ]:
dataset = load_dataset("samsum")

print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

In [ ]:
dataset['train'][0]

# Pre-process data

In [ ]:
# We will use the Flan T5 tokenizer to help us pre-process the data.
model_id="google/flan-t5-small"

# Load tokenizer of Flan-T5
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([
    dataset["train"], 
    dataset["test"]]).map(lambda x: tokenizer(x["dialogue"], truncation=True), 
    batched=True, 
    remove_columns=["dialogue", "summary"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([
    dataset["train"], 
    dataset["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), 
    batched=True, 
    remove_columns=["dialogue", "summary"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

In [ ]:
def preprocess_function(sample,padding="max_length"):

    # Add prefix to the input prompt for t5
    inputs = ["summarize: " + item for item in sample["dialogue"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, 
                             padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["summary"], 
                       max_length=max_target_length, padding=padding, 
                       truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels 
    # by -100 when we want to ignore padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, 
                                remove_columns=["dialogue", "summary", "id"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

# Base/ Foundation Model

In [ ]:
# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Fine-tune

In [ ]:
# We will use the Rouge metric for evaluations. ROUGE, or Recall-Oriented 
# Understudy for Gisting Evaluation, is a set of metrics and a software package 
# used for evaluating automatic summarization and machine translation software 
# in natural language processing. The metrics compare an automatically produced 
# summary or translation against a reference or a set of references 
# (human-produced) summary or translation.
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):

    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):

    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

In [ ]:
# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100

# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:
# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir="flan-t5-samsum",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=5,
    # logging & evaluation strategies
    logging_dir=f"flan-t5-samsum/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [ ]:
# Start training
trainer.train()

 # Evaluate

In [ ]:
# Evaluate using the same trainer.
trainer.evaluate()

# Try it out!

In [ ]:
# load model and tokenizer from with pipeline
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=0)

# select a random test sample
sample = dataset['test'][randrange(len(dataset["test"]))]
print(f"dialogue: \n{sample['dialogue']}\n---------------")
print(f"reference summary: \n{sample['summary']}\n---------------")

# summarize dialogue
result = summarizer(sample["dialogue"])

print(f"flan-t5-base summary:\n{result[0]['summary_text']}")